In [2]:
!pip install selenium

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.7 MB 5.6 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.7 MB 4.0 MB/s eta 0:00:02
   ------------ --------------------------- 3.1/9.7 MB 4.7 MB/s eta 0:00:02
   ------------------ --------------------- 4.5/9.7 MB 5.1 MB/s eta 0:00:02
   ----------------------- ---------------- 5.8/9.7 MB 5.3 MB/s eta 0:00:01
   ----------------------------- ---------- 7.1/9.7 MB 5.5 MB/s eta 0:00:01
   ----------------------------------- ---- 8.7/9.7 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 5.6 MB/s eta 0:00:00


In [4]:
!pip install tqdm

In [ ]:
from selenium.webdriver.common.by import By
import time
import json
import re
from tqdm import tqdm

def get_data(i, driver):
    # lhink trang
    link = f"https://batdongsan.com.vn/cho-thue-nha-tro-phong-tro-ha-noi/p{i}"

    try:
        # Mở trang danh sách
        driver.get(link)
        time.sleep(3)  # Đợi trang tải

        # Lấy danh sách các liên kết chi tiết (URLs)
        listings = driver.find_elements(By.CSS_SELECTOR, "a.js__product-link-for-product-id")
        links = [listing.get_attribute("href") for listing in listings]

        all_data = []  # Danh sách lưu dữ liệu các mục

        for link in tqdm(links, desc=f"Link: {link}", unit="link"):
            # Mở liên kết trong cùng tab
            driver.get(link)
            time.sleep(5)  # Đợi trang chi tiết tải

            # Lấy thông tin từ trang chi tiết
            try:
                title = driver.find_element(By.CSS_SELECTOR, "h1.re__pr-title").text
            except:
                title = ""

            try:
                rent_fee = driver.find_element(By.CSS_SELECTOR, "div.re__pr-short-info-item span.value").text.replace("triệu/tháng", "").replace(",", ".")
                rent_fee = float(rent_fee)
            except:
                rent_fee = ""

            try:
                detail_address = driver.find_element(By.CSS_SELECTOR, "span.js__pr-address").text
            except:
                detail_address = ""

            try:
                description = driver.find_element(By.CSS_SELECTOR, "div.re__detail-content").text
            except:
                description = ""

            try:
                acreage = driver.find_element(By.CSS_SELECTOR, "div.re__pr-short-info-item:nth-of-type(2) span.value").text
            except:
                acreage = ""

            try:
                contact_phone = driver.find_element(By.CSS_SELECTOR, "span.hidden-mobile").text.replace(" ", "")
            except:
                contact_phone = ""
            try:
                contact_name = driver.find_element(By.CSS_SELECTOR, "span.hidden-mobile").get_attribute("data-kyc-name")
            except:
                contact_name = ""
            
            try:
                image_div = driver.find_element(By.CSS_SELECTOR, "div.re__pr-image-cover")
                style_attribute = image_div.get_attribute("style")
                image_url = re.search(r'url\("(.*?)"\)', style_attribute).group(1)
            except:
                image_url = ""
            try:
                bread_crumb = driver.find_elements(By.CSS_SELECTOR, "div.re__breadcrumb a")
                city = bread_crumb[1].text
                district = bread_crumb[2].text
            except:
                city = ""
                district = ""



            # Thêm dữ liệu vào danh sách
            data = {
                "title": title,
                "rent_fee": rent_fee,
                "detail_address": detail_address,
                "description": description,
                "room_type": "MOTEL",
                "acreage": acreage,
                "url": link,
                "contact_phone": contact_phone,
                "contact_name": contact_name,
                "image_url": image_url,
                "city": city,
                "district": district
            }
            all_data.append(data)
            print("Getting data successfully at: ", title)


        # Lưu dữ liệu dưới dạng JSON
        with open(f"data/batdongsan_com_vn/listings_details_{i}.json", "w", encoding="utf-8") as f:
            json.dump(all_data, f, ensure_ascii=False, indent=4)

        print(f"Hoàn thành việc crawl và lưu dữ liệu tại page thứ {1}")

    except:
        pass



In [ ]:
from tqdm import tqdm
from selenium import webdriver
driver = webdriver.Chrome()
for i in tqdm(range(17), desc="Crawling first 8 pages", unit="page"):
        data = get_data(i + 1, driver)
# Đóng trình duyệt
driver.quit()

Crawling first 8 pages:   0%|          | 0/17 [00:00<?, ?page/s]

Getting data successfully at:  Cho thuê phòng trọ khép kín đầy đủ tiện nghi tại 173 phố Lò Đúc, quận Hai Bà Trưng, Hà Nội


Getting data successfully at:  Chính chủ cho thuê phòng dạng căn hộ mini 20m2 đủ đồ thoáng sáng tại Cầu Giấy 3,9tr/ tháng


Getting data successfully at:  Cho thuê phòng đẹp tại vị trí trung tâm 218 Đội Cấn


Getting data successfully at:  Ngõ 97 Phố Chính Kinh, chung cư full đồ, đầy đủ PCCC, gần rất nhiều trường đại học (NV, TN... )


...

Getting data successfully at:  Ngõ 190 Nguyễn Trãi, chung cư full đồ, đầy đủ PCCC, gần rất nhiều trường đại học, Lh 0904 253 ***


Getting data successfully at:  Chính chủ cho thuê phòng MP Đê La Thành: 25m2, điều hòa, nóng lạnh, thang máy, giá: 3,5tr/tháng


Getting data successfully at:  Chính chủ cho thuê phòng CCMN ở Đội Cấn. Khu vực Vip của quận Ba Đình, giáp quận Đống Đa, Cầu Giấy


Getting data successfully at:  Cho thuê CHDV 1 khách + 1 ngủ Đình Thôn


Getting data successfully at:  Cho thuê nhà trọ full nội thất, đường 1A, 30m2, 2,5 triệu giá ưu đãi
